# Format Spec Mini-Language

Deep dive into the format spec syntax: custom keys, render hints, and advanced patterns.

In [1]:
from t_prompts import prompt

## Format Spec Syntax

The format spec follows the pattern: `{expression:key:render_hints}`

- `expression`: The Python expression to interpolate
- `key`: The key for accessing this interpolation (defaults to expression)
- `render_hints`: Optional metadata for custom renderers (everything after the second `:`)

In [2]:
# No format spec: key is the expression
user_query = "What is Python?"
p1 = prompt(t"{user_query}")
print(f"No format spec - Key: {list(p1.keys())[0]}")

# With key only
p2 = prompt(t"{user_query:query}")
print(f"With key - Key: {list(p2.keys())[0]}")

# With key and render hints
p3 = prompt(t"{user_query:query:format=json}")
print(f"With hints - Key: {list(p3.keys())[0]}")
print(f"With hints - Render hints: {p3['query'].render_hints}")

No format spec - Key: user_query
With key - Key: query
With hints - Key: query
With hints - Render hints: format=json


## Using Underscore for Expression as Key

Use `_` to explicitly use the expression as the key (useful when you want to add render hints without changing the key).

In [3]:
# Underscore: use expression as key
value = "test"
p = prompt(t"{value:_}")
print(f"Key: {list(p.keys())[0]}")
assert 'value' in p

# Note: When using render hints, you need to specify the key explicitly
# The underscore only works when it's the only thing in the format spec
p2 = prompt(t"{value:value:uppercase}")
print(f"\nKey with hints: {list(p2.keys())[0]}")
print(f"Render hints: {p2['value'].render_hints}")

Key: value

Key with hints: value
Render hints: uppercase


## Render Hints for Custom Behavior

Render hints are stored in the `render_hints` field and can be parsed by custom renderers. They're not used by the default renderer.

In [4]:
data = '{"name": "Alice"}'
p = prompt(t"{data:user_data:format=json,indent=2}")

node = p['user_data']
print(f"Key: {node.key}")
print(f"Render hints: {node.render_hints}")
print(f"Value: {node.value}")

Key: user_data
Render hints: format=json,indent=2
Value: {"name": "Alice"}


## Parsing Render Hints

You can parse render hints however you like. Here's an example of parsing key-value pairs.

In [5]:
def parse_hints(hints):
    """Parse render hints as comma-separated key=value pairs."""
    if not hints:
        return {}
    result = {}
    for pair in hints.split(','):
        if '=' in pair:
            key, value = pair.split('=', 1)
            result[key.strip()] = value.strip()
    return result

# Example with multiple hints
content = "<html><body>Hello</body></html>"
p = prompt(t"{content:html:format=xml,indent=4,preserve_whitespace=true}")

hints = parse_hints(p['html'].render_hints)
print("Parsed hints:")
for key, value in hints.items():
    print(f"  {key}: {value}")

Parsed hints:
  format: xml
  indent: 4
  preserve_whitespace: true


## List Separator Hint

The `sep=<value>` render hint is recognized by the default renderer for list interpolations.

In [6]:
items = [
    prompt(t"{item:item}")
    for item in ["apple", "banana", "cherry"]
]

# Default separator (newline)
p1 = prompt(t"{items:items}")
print("Default separator:")
print(str(p1))

# Custom separator
p2 = prompt(t"{items:items:sep=, }")
print("\nCustom separator ', ':")
print(str(p2))

Default separator:
apple
banana
cherry

Custom separator ', ':
apple, banana, cherry


## Dynamic Format Specs (Advanced)

Format specs can contain interpolated values, enabling programmatic key generation.

In [7]:
# Generate keys programmatically
prefix = "config"
section = "database"
value = "postgresql://localhost"

p = prompt(t"{value: {prefix}_{section} }")
print(f"Generated key: {list(p.keys())[0]}")
print(f"Value: {p['config_database'].value}")

Generated key: config_database
Value: postgresql://localhost


## Complex Format Spec Example

Combine dynamic keys with render hints for maximum flexibility.

In [8]:
# Build a prompt with dynamic keys and render hints
section = "user_profile"
field = "bio"
content = "Software engineer interested in AI."

p = prompt(t"{content: {section}_{field} :max_length=100,truncate=true}")

node = list(p.values())[0]
print(f"Key: {node.key}")
print(f"Render hints: {node.render_hints}")
print(f"Value: {node.value}")

# Parse the hints
hints = parse_hints(node.render_hints)
print("\nParsed hints:")
for k, v in hints.items():
    print(f"  {k}: {v}")

Key: user_profile_bio
Render hints: max_length=100,truncate=true
Value: Software engineer interested in AI.

Parsed hints:
  max_length: 100
  truncate: true
